In [58]:
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import glob
import time
import undetected_chromedriver as uc

In [100]:
# chrome_options = webdriver.ChromeOptions()
# prefs = {'download.default_directory' : os.getcwd()}
#chrome_options.add_argument("--headless")
# chrome_options.add_experimental_option('prefs', prefs)
# driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
# service = ChromeService(executable_path=ChromeDriverManager().install())
driver = uc.Chrome(headless=True,use_subprocess=True)


In [60]:
def openBudgetSite(driver, school_code, fiscal_year, budgetOrAllocation = 'budget'):
    if (budgetOrAllocation == 'allocation'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxyallocation/default.aspx")

    elif (budgetOrAllocation == 'budget'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxybudgetsummaryto/default.aspx")
        
    else:
        raise ValueError('Budget or Allocation value must be either "budget" or "allocation"')
                         
    # get element 
    element = driver.find_element(By.XPATH,'//*[@id="School_Code"]')

    # send keys 
    element.send_keys(school_code)

    x = driver.find_element(By.XPATH,'//*[@id="Fiscal_Year"]')
    drop=Select(x)

    # select by visible text
    drop.select_by_visible_text(fiscal_year)

    driver.find_element(By.XPATH,'//*[@id="Enter"]').click()
    

    try:
        element = driver.find_element(By.XPATH,'//*[@id="message"]/div[1]/div[3]/h2/a')
        element.text.index(school_code)
        return True

    except NoSuchElementException:
        print('No ' + budgetOrAllocation + ' data could be found for school code ' + school_code + ' in the year ' + year)
        return False
        
    except ValueError:
        print('Given School code and school code for retrieved data do not match')
        return False


def allocationPageScraper(driver, school_code, fiscal_year):
    list_of_dfs = pd.read_html(driver.page_source)
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'allocation_category', 'amount'])

    for df in list_of_dfs:
        df = df.rename(columns={df.columns[0]: 'allocation_category', df.columns[1]: 'amount'})
        df = df[df["allocation_category"].str.contains("total", case = False)==False]
        df['location_code'] = school_code
        df['fiscal_year'] = fiscal_year
        final_df = pd.concat([final_df, df])
    
    return final_df
    
    
def budgetPageScraper(driver, school_code, fiscal_year):
    section_titles = driver.find_elements(By.CLASS_NAME , 'TO_Section')
    list_of_dfs = pd.read_html(driver.page_source)
    
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'budget_category', 'budget_assignment', 'num_positions', 'service_type', 'amount'])

    for i,df in enumerate(list_of_dfs):
        if (len(df.columns) == 2):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'amount'})
            df["budget_assignment"] = df["budget_assignment"].astype(str)
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['num_positions'] = None
            df['service_type'] = None
            final_df = pd.concat([final_df, df])
        
        elif (len(df.columns) == 3):
            if (section_titles[i].text == 'School Site'):
                continue
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'num_positions', df.columns[2]: 'amount'})
            df["budget_assignment"] = df["budget_assignment"].astype(str)
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['service_type'] = None
            final_df = pd.concat([final_df, df])

        elif (len(df.columns) == 4):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'service_type', df.columns[2]: 'num_positions', df.columns[3]: 'amount'})
            df["budget_assignment"] = df["budget_assignment"].astype(str)
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df = df[df["service_type"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            final_df = pd.concat([final_df, df])
            
    return final_df

In [61]:
if (openBudgetSite(driver, 'M125', '2022', 'budget')):
    df = budgetPageScraper(driver, 'M125', '2022')

In [62]:
df

,location_code,fiscal_year,budget_category,budget_assignment,num_positions,service_type,amount
0,M125,2022,Leadership,HEAD OF SCHOOL,1.0,None,"$ 84,471"
1,M125,2022,Leadership,AP - SUPERVISION,1.0,None,"$ 121,384"
0,M125,2022,Coordinator/Supervisor/Dean,PARENT COORDINATOR,1.0,None,"$ 53,241"
0,M125,2022,Secretary,School Secretary,1.0,None,"$ 76,956"
0,M125,2022,Classroom Teacher,Pre K,2.0,WHOLE CLASS,"$ 178,549"
1,M125,2022,Classroom Teacher,K,2.0,WHOLE CLASS,"$ 207,632"
2,M125,2022,Classroom Teacher,"K,1",1.0,SELF-CONTAINED SP ED,"$ 97,194"
3,M125,2022,Classroom Teacher,1,2.0,WHOLE CLASS,"$ 198,553"
4,M125,2022,Classroom Teacher,2,2.0,WHOLE CLASS,"$ 194,824"
5,M125,2022,Classroom Teacher,3,2.0,WHOLE CLASS,"$ 231,099"


In [63]:
df

,location_code,fiscal_year,budget_category,budget_assignment,num_positions,service_type,amount
0,M125,2022,Leadership,HEAD OF SCHOOL,1.0,None,"$ 84,471"
1,M125,2022,Leadership,AP - SUPERVISION,1.0,None,"$ 121,384"
0,M125,2022,Coordinator/Supervisor/Dean,PARENT COORDINATOR,1.0,None,"$ 53,241"
0,M125,2022,Secretary,School Secretary,1.0,None,"$ 76,956"
0,M125,2022,Classroom Teacher,Pre K,2.0,WHOLE CLASS,"$ 178,549"
1,M125,2022,Classroom Teacher,K,2.0,WHOLE CLASS,"$ 207,632"
2,M125,2022,Classroom Teacher,"K,1",1.0,SELF-CONTAINED SP ED,"$ 97,194"
3,M125,2022,Classroom Teacher,1,2.0,WHOLE CLASS,"$ 198,553"
4,M125,2022,Classroom Teacher,2,2.0,WHOLE CLASS,"$ 194,824"
5,M125,2022,Classroom Teacher,3,2.0,WHOLE CLASS,"$ 231,099"


In [101]:
# school_data = pd.read_csv(r'C:\Users\ebroh\Downloads\2019_-_2020_School_Locations (2).csv')
school_data = pd.read_csv("../school_code_budget_extraction.csv")

In [65]:
# school_data['school_code'].unique() 

array(['M040', 'M142', 'K953', ..., 'X723', 'X754', 'X811'], dtype=object)

In [102]:
import time

budget_data = pd.read_csv('../data/budget_all.csv')
allocation_data =  pd.read_csv('../data/allocation_all.csv')
years = ['2018', '2019', '2020', '2021', '2022', '2023']
c=0
for index, row in school_data.iterrows():
    if (not pd.isnull(row["school_code"])) and row["2023"] != "done" : #some of the rows in school data are unfortunately empty, so just skip them. AND if the row is not Completely "done" yet then do it whole again
        c+=1
        if c==300:
            time.sleep(1800) # sleep for 30 mins 
            driver = uc.Chrome(headless=True,use_subprocess=True)
            c=0
        print(row)
        school = row["school_code"]
        for year in years:
            if (openBudgetSite(driver, school, year, 'budget')): # ToDo: split budget from allocation and use 2 different drivers to reduce rate limit and parallelize
                print(year +  ' Budget Data Opened')
                budget_data = pd.concat([budgetPageScraper(driver, school, year), budget_data])
            if (openBudgetSite(driver, school, year, 'allocation')):
                print(year +  ' Allocation Data Opened')
                allocation_data = pd.concat([allocationPageScraper(driver, school, year), allocation_data])
            school_data[year][index]="done"


school_code    M314
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 304, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    M319
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 305, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    M322
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 306, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    M324
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 307, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    M328
2018           done
2019           done
2020           done
2021           done
2022           done
2023        

2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X277
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 343, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X296
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 344, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X298
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 345, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X334
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 346, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X343
2018           done
2019           done
2020           done


2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X146
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 382, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X152
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 383, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X182
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 384, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X269
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 385, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X282
2018           done
2019           done
2020           done


2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X042
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 420, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X053
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 421, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X055
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 422, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X058
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 423, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X063
2018           done
2019           done
2020           done


2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X323
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 459, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X324
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 460, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X327
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 461, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X328
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 462, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    X329
2018           done
2019           done
2020           done


...

2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q152
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1429, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q166
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1430, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q171
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1431, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q204
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1432, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q222
2018           done
2019           done
2020           d

2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R018
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1468, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R019
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1469, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R020
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1470, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R021
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1471, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R022
2018           done
2019           done
2020           d

2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R061
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1507, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R062
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1508, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R063
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1509, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R064
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1510, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    R065
2018           done
2019           done
2020           d

2023 Budget Data Opened
2023 Allocation Data Opened
school_code    K383
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1546, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    K384
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1547, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    K403
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1548, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    K545
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1549, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    K549
2018           done
2019           done
2020           d

2023 Budget Data Opened
2023 Allocation Data Opened
school_code    M811
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1585, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q004
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1586, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q009
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1587, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q023
2018           done
2019           done
2020           done
2021           done
2022           done
2023            NaN
Name: 1588, dtype: object
2023 Budget Data Opened
2023 Allocation Data Opened
school_code    Q075
2018           done
2019           done
2020           d

In [103]:
budget_data.to_csv('../data/budget_all.csv',index=False)

In [104]:
allocation_data.to_csv("../data/allocation_all.csv",index=False)

In [105]:
school_data.to_csv("../school_code_budget_extraction.csv",index=False)

In [ ]:
### Note: for missing schools i.e. schools with no data in budget_all.csv and allocation_call_csv its possible something went wrong.
# for this, the school_code_budget_extraction should show all dates marked "done" but nothing in the main sheets.